# Betting the NFL Over/Under - A Data Science Perspective
## Scraper 1 - Acquire Game Identifiers

**This scraper accesses the gamelogs of every NFL team to determine the urls from which the game data will be obtained. This scraper, to be used for years 2010-2019, works one year at a time. The gamelogs and dataframes to be used in Scraper 2 are stored in binary files.</font>** 

**First, the necessary packages are loaded, and the structure of the dataframe that will be populated by the acquired data is set up.</font>** 

In [ ]:
import requests
import re
import pandas as pd
import datetime as dt
import pickle
from dateutil.parser import parse

In [3]:
teams = {'Atlanta Falcons':'atl','Buffalo Bills':'buf','Carolina Panthers':'car','Chicago Bears':'chi',
         'Cincinnati Bengals':'cin','Cleveland Browns':'cle','Indianapolis Colts':'clt',
         'Arizona Cardinals':'crd','Dallas Cowboys':'dal','Denver Broncos':'den','Detroit Lions':'det',
         'Green Bay Packers':'gnb','Houston Texans':'htx','Jacksonville Jaguars':'jax',
         'Kansas City Chiefs':'kan','Miami Dolphins':'mia','Minnesota Vikings':'min','New Orleans Saints':'nor',
         'New England Patriots':'nwe','New York Giants':'nyg','New York Jets':'nyj','Tennessee Titans':'oti',
         'Philadelphia Eagles':'phi','Pittsburgh Steelers':'pit','Oakland Raiders':'rai',
         'Las Vegas Raiders':'rai','St. Louis Rams':'ram','Los Angeles Rams':'ram','Baltimore Ravens':'rav',
         'San Diego Chargers':'sdg','Los Angeles Chargers':'sdg','Seattle Seahawks':'sea',
         'San Francisco 49ers':'sfo','Tampa Bay Buccaneers':'tam','Washington Redskins':'was'}

# "org" is the collection of team abbreviations in the form of a set as due to changes in team names, 
# multiples team names share common organization, abbreviation. A set gives the unique values.
org = set(teams.values())

# Index and columns to be used for "dfyear" multiindex dataframe with team abbreviations as the primary index 
# and row number as the temporary secondary index.
yearcols = ['Code','Home/Away','Opponent','Points','Points_Opp','Yds_Off_Pass','Yds_Off_Rush','Yds_Def_Pass',
            'Yds_Def_Rush','TD','TD_on_Def','FG_Made','FG_Att','RZ_Conv','RZ_Att','RZ_Def_Conv','RZ_Def_Att',
            'Possession','Plays','TO_Gained','TO_Lost','Yds_Pen','Sacks_Def','Tackles_Loss','Yds_per_Kickret',
            'Yds_per_Puntret']
ind = pd.MultiIndex.from_product([org,[i for i in range(16)]],names=['Team','row'])

***table* is the raw dataframe obtained from the website. *dfteam* is a preliminary dataframe for a given team and given year used to populate the larger *dfyear* dataframe. It stores the parsed game dates as well as the home/away and opponent columns. These columns are used to formulate the contest identifiers (gamecodes), stored in a list, which will be used to access the urls necessary to obtain the bulk of the data.**

In [4]:
years = [str(i) for i in range(2010,2020)]
gcodes = []
for year in years:
    dfyear=pd.DataFrame(index=ind,columns=yearcols)
    for team in org:
        url = 'https://www.pro-football-reference.com/teams/'+team+'/'+year+'/gamelog/'
        res = requests.get(url)        

        tableID = 'gamelog'+year
        table=pd.read_html(res.text,attrs={'id':tableID},flavor='bs4')
        table=table[0]

# Columns for "table" are renamed for ease in populating our dataframe     

        L0 = list(table.columns.get_level_values(0))
        L0_mod = ['Event' if 'Unnamed' in L0[i] else L0[i] for i in range(len(L0))]
        L0_renmd = dict(zip(L0,L0_mod))
        table.rename(columns=L0_renmd,level=0,inplace=True)

        dfteam = pd.DataFrame(columns=['Date','Home/Away','Opponent'])

# String dates from "table" are converted to numerical in proper format for use in url
# Special consideration for Janaury, as those dates correspond to following year
        dconv = lambda x: (parse(x + ' ' + str(int(year)+1)).strftime('%Y%m%d') 
                           if 'January' in x else parse(x + ' ' + year).strftime('%Y%m%d'))
        dfteam['Date'] = table['Event']['Date'].apply(dconv)

# Home and Away designation determines how gamecode is labeled    
        dfteam['Home/Away'][table.iloc[:,6] == '@'] = 'A'
        dfteam['Home/Away'][table.iloc[:,6] != '@'] = 'H'
        dfteam['Opponent'] = table['Event']['Opp']

# Because of "dfyear" multiindex, cannot simply copy "dfteam" dataframe to "dfyear". Must use .values        
        dfyear.loc[team,'Home/Away':'Opponent'] = dfteam[['Home/Away','Opponent']].values

# Gamecodes, the contest identifier, are determined to put in list for later url access 
# and to put in dfyear dataframe for game identification
        for row in range(16):
            date = dfteam['Date'][row]
            if dfteam['Home/Away'][row] == 'H':
                hometeam = team
            elif dfteam['Home/Away'][row] == 'A':
                hometeam = teams[dfteam['Opponent'][row]]
            gamecode = date+'0'+hometeam
            dfyear.loc[(team,row),'Code'] = gamecode
            if gamecode not in gcodes: gcodes.append(gamecode)

# Replace row with gamecode in index
    dfyear=dfyear.reset_index(level=1,drop=True)
    dfyear=dfyear.set_index([dfyear.index,dfyear['Code']])
    dfyear.drop('Code',axis=1,inplace=True)

# Check for null values, number of gcodes matches with number of games in season (32*16/2=256).
#    print(dfyear.loc[:,'Home/Away':'Opponent'].isnull().sum())
#    print(len(gcodes))

# Store gamecodes and working dataframe in binary files
    path = 'data/df_step1_'+year+'.data'
    with open(path, 'wb') as f:
        pickle.dump(dfyear,f)

In [ ]:
path = 'data/gamecodes.data'
with open(path, 'wb') as f:
    pickle.dump(gcodes,f)

## Scraper 2 - Acquire Game Data

**Define functions to use in data scraping**

In [ ]:
from datetime import time, timedelta

In [ ]:
def pbp_to_tdelt(time):
    if ':' in time:
        elem = pd.to_numeric(time.split(':'))
        time = pd.to_timedelta(elem[0],unit='m') + pd.to_timedelta(elem[1],unit='s')
        return time
    else:
        return time

In [ ]:
def neg_check(strng,idx):
    elem = strng.split('-')
    if elem[idx] == '':
        del(elem[idx])
        elem[idx] = '-'+elem[idx]
    return elem

In [ ]:
# Extract table 'tid' from game 'code'
def get_table(res,code,tid):
    import re
    import pandas as pd
    
    comm = re.compile('<!--|-->')
    try:
        table = pd.read_html(comm.sub("",res.text),attrs={'id':tid},flavor='bs4')
    except ValueError:
        print(f'Problem with {tid} table in game {code}. Please check!')
        return pd.DataFrame()
    else:
        return table[0]

In [ ]:
# Get list of game identifiers from file
def get_codes(path):
    with open(path, 'rb') as f:
        codes = pickle.load(f)
    return codes

In [ ]:
# Extract year from game identifier
def get_year(code):
    if (code[4:6] == '01'):
        year = str(int(code[0:4])-1)
    else:
        year = str(int(code[0:4]))
    return year

In [ ]:
# Get subset of games from year 'yr'
def year_codes(codes,yr):
    subset = [codes[i] for i in range(len(codes)) if get_year(codes[i]) == yr]
    return subset

**Retrieve all game identifiers (gamecodes) and choose year to subset**

In [ ]:
year = '2018'
path = 'data/gamecodes.data'
gcodes = get_codes(path)
gc = year_codes(gcodes,year)

**Retrieve dataframe to be filled in with initial data**

In [ ]:
path = 'data/df_step1_'+year+'.data'
with open(path, 'rb') as f:
    dfyear = pickle.load(f)

**Scrape data from website**

In [ ]:
tableids = ['game_info','team_stats','player_defense','returns','kicking','home_drives','vis_drives','pbp']
for gamecode in gc:
    wpath = 'data/raw/'+year+'/'+gamecode+'.pkl'
    f = open(wpath, 'wb')
    url = 'https://www.pro-football-reference.com/boxscores/'+gamecode+'.htm'
    res = requests.get(url)
    table = pd.read_html(res.text,attrs={'class':'linescore nohover stats_table no_freeze'},flavor='bs4')
    pickle.dump(table[0],f)
    for tableid in tableids:
        table = get_table(res,gamecode,tableid)
        pickle.dump(table,f)
    f.close()
print('End')

**Fill in *dfyear* dataframe with data from tables acquired from web scraping**

In [ ]:
for gamecode in gc:
    rpath = 'data/raw/'+year+'/'+gamecode+'.pkl'
    f = open(rpath, 'rb')
    
# Team designation
    table = pickle.load(f)
    awayteam = raw.iloc[0,1]
    away = teams[awayteam]
    hometeam = table.iloc[1,1]
    home = teams[hometeam]
    
# Points
    dfyear.loc[(away,code),'Points'] = int(table.loc[0,'Final'])
    dfyear.loc[(home,gamecode),'Points'] = int(table.loc[1,'Final'])
    dfyear.loc[(away,gamecode),'Points_Opp'] = int(table.loc[1,'Final'])
    dfyear.loc[(home,gamecode),'Points_Opp'] = int(table.loc[0,'Final'])
    
# Offensive & Defensive Rushing Yards
    table = pickle.load(f)
    table = pickle.load(f)
    rush1 = neg_check(table.iloc[1,1],1) # Check for negative rushing yards
    dfyear.loc[(away,gamecode),'Yds_Off_Rush'] = int(rush1[1])
    dfyear.loc[(home,gamecode),'Yds_Def_Rush'] = int(rush1[1])
    rush2 = neg_check(table.iloc[1,2],1) # Check for negative rushing yards
    dfyear.loc[(home,gamecode),'Yds_Off_Rush'] = int(rush2[1])
    dfyear.loc[(away,gamecode),'Yds_Def_Rush'] = int(rush2[1])
    
# Offensive & Defensive Passing Yards
    pass1 = neg_check(table.iloc[2,1],2) # Check for negative passing yards
    dfyear.loc[(away,gamecode),'Yds_Off_Pass'] = int(pass1[2])
    dfyear.loc[(home,gamecode),'Yds_Def_Pass'] = int(pass1[2])
    pass2 = neg_check(table.iloc[2,2],2) # Check for negative passing yards
    dfyear.loc[(home,gamecode),'Yds_Off_Pass'] = int(pass2[2])
    dfyear.loc[(away,gamecode),'Yds_Def_Pass'] = int(pass2[2])
    
# Time of Possession  (Converted to fractional minutes)  
    dfyear.loc[(away,gamecode),'Possession'] = \
        float(table.iloc[11,1].split(':')[0])+float(table.iloc[11,1].split(':')[1])/60.0
    dfyear.loc[(home,gamecode),'Possession'] = \
        float(table.iloc[11,2].split(':')[0])+float(table.iloc[11,2].split(':')[1])/60.0
    
# Penalties
    dfyear.loc[(away,gamecode),'Yds_Pen'] = int(table.iloc[8,1].split('-')[1])
    dfyear.loc[(home,gamecode),'Yds_Pen'] = int(table.iloc[8,2].split('-')[1])
    
# Defensive Sacks
    dfyear.loc[(away,gamecode),'Sacks_Def'] = int(table.iloc[3,2].split('-')[0])
    dfyear.loc[(home,gamecode),'Sacks_Def'] = int(table.iloc[3,1].split('-')[0])
    
# Turnovers
    dfyear.loc[(away,gamecode),'TO_Gained'] = int(table.iloc[7,2])
    dfyear.loc[(home,gamecode),'TO_Gained'] = int(table.iloc[7,1])
    dfyear.loc[(away,gamecode),'TO_Lost'] = int(table.iloc[7,1])
    dfyear.loc[(home,gamecode),'TO_Lost'] = int(table.iloc[7,2])
    
# Touchdowns (Defensive & Special Teams to be added later)
    dfyear.loc[(away,gamecode),'TD'] = int(rush1[2])+int(pass1[3])
    dfyear.loc[(home,gamecode),'TD'] = int(rush2[2])+int(pass2[3])
    
# Defense
    table = pickle.load(f)
# Remove multilevel, NANs, separator row
    table = table.droplevel(level=0,axis=1)
    table = table.dropna(subset=['Tm']).iloc[:,1:]
    table = table[table.iloc[:,0]!='Tm']

# Rename columns to remove column name duplication
    cols = list(table.columns)
    cols[3] = 'TD_Int'
    cols[2] = 'Yds_Int'
    cols[13] = 'Yds_Fum'
    cols[14] = 'TD_Fum'
    table.columns = cols

# Change value types to numeric types
    cols = [list(table.columns[1:6]),table.columns[6],list(table.columns[7:])]
    for grp in cols:
        table[grp]=table[grp].apply(pd.to_numeric)
        
# Group by team to calculate team values        
    grouped = table.groupby(table['Tm'],sort=False)
    
# Add Defensive Touchdowns
    dftd=grouped['TD_Int'].sum()+grouped['TD_Fum'].sum()
    dfyear.loc[(away,gamecode),'TD'] += dftd[0]
    dfyear.loc[(home,gamecode),'TD'] += dftd[1]
    dfyear.loc[(away,gamecode),'TD_on_Def'] = dftd[0]
    dfyear.loc[(home,gamecode),'TD_on_Def'] = dftd[1]
    
# Tackles for Loss
    tfl=grouped['TFL'].sum()
    dfyear.loc[(away,gamecode),'Tackles_Loss'] = tfl[0]
    dfyear.loc[(home,gamecode),'Tackles_Loss'] = tfl[1]

# Kick/Punt Returns
    table = pickle.load(f)
    if not(table.empty):
        table = table.droplevel(level=0,axis=1)
        table = table.dropna(subset=['Tm']).iloc[:,1:]
        table = table[table.iloc[:,0]!='Tm']
        cols = list(table.columns)
        cols[2] = 'Yds_KR'
        cols[7] = 'Yds_PR'
        cols[5] = 'Lng_KR'
        cols[10] = 'Lng_PR'
        cols[4] = 'TD_KR'
        cols[9] = 'TD_PR'
        table.columns = cols
        cols = [list(table.columns[1:3]),table.columns[3],list(table.columns[4:8]),table.columns[8],
                list(table.columns[9:])]
        for grp in cols:
            table[grp]=table[grp].apply(pd.to_numeric)

        grouped = table.groupby(table['Tm'],sort=False)
        kptd=grouped['TD_KR'].sum()+grouped['TD_PR'].sum()
        kyds=grouped['Yds_KR'].sum()
        knum=grouped['Rt'].sum()
        pyds=grouped['Yds_PR'].sum()
        pnum=grouped['Ret'].sum()
        if (kptd.shape[0] == 0):
            a_kptd = h_kptd = a_knum = h_knum = a_kyds = h_kyds = a_pyds = h_pyds = a_pnum = h_pnum = 0
        elif (kptd.shape[0] == 1):
            if (away.upper() == table.loc[0,'Tm']):
                a_kptd = kptd[0]
                a_knum = knum[0]
                a_kyds = kyds[0]
                a_pyds = pyds[0]
                a_pnum = pnum[0]
                h_kptd = h_knum = h_kyds = h_pyds = h_pnum = 0
            elif (home.upper() == table.loc[0,'Tm']):
                h_kptd = kptd[0]
                h_knum = knum[0]
                h_kyds = kyds[0]
                h_pyds = pyds[0]
                h_pnum = pnum[0]
                a_kptd = a_knum = a_kyds = a_pyds = a_pnum = 0
        else:
                a_kptd = kptd[0]
                a_knum = knum[0]
                a_kyds = kyds[0]
                a_pyds = pyds[0]
                a_pnum = pnum[0]
                h_kptd = kptd[1]
                h_knum = knum[1]
                h_kyds = kyds[1]
                h_pyds = pyds[1]
                h_pnum = pnum[1]
        dfyear.loc[(away,gamecode),'TD'] += a_kptd
        dfyear.loc[(home,gamecode),'TD'] += h_kptd
        if (a_knum > 0):
            dfyear.loc[(away,gamecode),'Yds_per_Kickret'] = float(a_kyds)/float(a_knum)
        else:
            dfyear.loc[(away,gamecode),'Yds_per_Kickret'] = 0.0
        if (h_knum > 0):
            dfyear.loc[(home,gamecode),'Yds_per_Kickret'] = float(h_kyds)/float(h_knum)
        else:
            dfyear.loc[(home,gamecode),'Yds_per_Kickret'] = 0.0
        if (a_pnum > 0):
            dfyear.loc[(away,gamecode),'Yds_per_Puntret'] = float(a_pyds)/float(a_pnum)
        else:
            dfyear.loc[(away,gamecode),'Yds_per_Puntret'] = 0.0
        if (h_pnum > 0):
            dfyear.loc[(home,gamecode),'Yds_per_Puntret'] = float(h_pyds)/float(h_pnum)
        else:
            dfyear.loc[(home,gamecode),'Yds_per_Puntret'] = 0.0

# Field Goals
    table = pickle.load(f)
    table = table.droplevel(level=0,axis=1)
    table = table.dropna(subset=['Tm']).iloc[:,1:]
    table = table[table.iloc[:,0]!='Tm']
    cols = [list(table.columns[1:7]),table.columns[7],table.columns[8]]
    for grp in cols:
        table[grp]=table[grp].apply(pd.to_numeric)
    grouped = table.groupby(table['Tm'],sort=False)
    fgm=grouped['FGM'].sum()
    fga=grouped['FGA'].sum()
    dfyear.loc[(away,gamecode),'FG_Made'] = fgm[0]
    dfyear.loc[(home,gamecode),'FG_Made'] = fgm[1]
    dfyear.loc[(away,gamecode),'FG_Att'] = fga[0]
    dfyear.loc[(home,gamecode),'FG_Att'] = fga[1]
    
# Total Plays & Set up Drive and Play-by-Play tables for Red Zone calculation
    table = pickle.load(f)
    hdrive = table.iloc[:,1:]
    hdrive.index = range(1,len(hdrive)+1)
    hdrive.index.set_names('Drive',inplace=True)
    hdrive.rename({'Net_yds':'Yds'},axis=1,inplace=True)
    dfyear.loc[(home,gamecode),'Plays'] = hdrive['Plays'].sum()
    table = pickle.load(f)
    adrive = table.iloc[:,1:]
    adrive.index = range(1,len(adrive)+1)
    adrive.index.set_names('Drive',inplace=True)
    adrive.rename({'Net_yds':'Yds'},axis=1,inplace=True)
    dfyear.loc[(away,gamecode),'Plays'] = adrive['Plays'].sum()

    hdrive['Quarter']=hdrive['Quarter'].fillna(method='ffill')
    adrive['Quarter']=adrive['Quarter'].fillna(method='ffill')
    hdrive['LOS']=hdrive['LOS'].fillna(home.upper()+' 25')
    position=hdrive['LOS'].str.split(expand=True)
    hdrive.drop('LOS',axis=1,inplace=True)
    hdrive[['Side','Marker']] = position
    adrive['LOS']=adrive['LOS'].fillna(away.upper()+' 25')
    position=adrive['LOS'].str.split(expand=True)
    adrive.drop('LOS',axis=1,inplace=True)
    adrive[['Side','Marker']] = position
    cols=['Quarter','Time','Side','Marker','Plays','Length','Yds','Result']
    hdrive[hdrive['Side']=='50']=hdrive[hdrive['Side']=='50'].assign(Marker = '50') # Case where ball is on 50 yard line
    hdrive[hdrive['Side']=='50']=hdrive[hdrive['Side']=='50'].assign(Side = home.upper())
    hdrive=hdrive.reindex(columns=cols,copy=False)
    adrive[adrive['Side']=='50']=adrive[adrive['Side']=='50'].assign(Marker = '50') # Case where ball is on 50 yard line
    adrive[adrive['Side']=='50']=adrive[adrive['Side']=='50'].assign(Side = home.upper())
    adrive=adrive.reindex(columns=cols,copy=False)

    intcols=['Plays','Yds','Quarter','Marker']
    timecols=['Time','Length']
    strcols=['Result','Side']

    for colu in intcols:
        hdrive[colu] = hdrive[colu].astype('int')
        adrive[colu] = adrive[colu].astype('int')
    for colu in strcols:
        hdrive[colu] = hdrive[colu].astype('str')
        adrive[colu] = adrive[colu].astype('str')
    for colu in timecols:
        for drive in range(1,hdrive.shape[0]+1):
            entry = hdrive.loc[drive,colu].split(':')
            hdrive.loc[drive,colu] = timedelta(hours=0,minutes=int(entry[0]),seconds=int(entry[1]))
        for drive in range(1,adrive.shape[0]+1):
            entry = adrive.loc[drive,colu].split(':')
            adrive.loc[drive,colu] = timedelta(hours=0,minutes=int(entry[0]),seconds=int(entry[1]))
    table = pickle.load(f)
    f.close()
    pbp=table.iloc[:,0:6]
    if (pbp.columns.nlevels > 1):                 # Sometimes multilevel object is retrieved. Ensure that
        pbp.columns = pbp.columns.droplevel()     # columns are single level
    pbp=pbp.dropna()
    pbp=pbp[pbp['Time'].str.contains(pat=':',regex=False)]
    pbp['Time'] = pbp['Time'].apply(pbp_to_tdelt)
    position=pbp['Location'].str.split(expand=True)
    position=position.iloc[:,0:2]
    pbp=pbp.drop('Location',axis=1)
    pbp[['Team','Location']] = position
    cols=['Quarter','Time','Down','ToGo','Team','Location','Detail']
    pbp=pbp.reindex(columns=cols,copy=False)
    pbp=pbp.reset_index(drop=True)
    pbp[pbp['Team']=='50']=pbp[pbp['Team']=='50'].assign(Location = '50') # Case where ball is on 50 yard line
    pbp[pbp['Team']=='50']=pbp[pbp['Team']=='50'].assign(Team = home.upper())
    pbp['Location']=pbp['Location'].astype('int')    
    pbp[pbp['Quarter']=='OT']=pbp[pbp['Quarter']=='OT'].assign(Quarter = '5') # Case of Overtime
    pbp['Quarter']=pbp['Quarter'].astype('int')

# Red Zone Routine
    team = {'H':home, 'A':away}
    drive = {home:1, away:1}
    df = {home:hdrive, away:adrive}
    rz_conv = {home:0, away:0}
    rz_att = {home:0, away:0}
    row = 0
#  Red Zone starts at 20 yard line
    while ((drive[home] <= hdrive.shape[0]) | (drive[away] <= adrive.shape[0])):
        stop = False
        if ((drive[home] <= hdrive.shape[0]) & (drive[away] <= adrive.shape[0])):
            if (hdrive.loc[drive[home],'Quarter'] < adrive.loc[drive[away],'Quarter']):
                poss = 'H'
                starttime = hdrive.loc[drive[home],'Time']
            elif (adrive.loc[drive[away],'Quarter'] < hdrive.loc[drive[home],'Quarter']):
                poss = 'A'
                starttime = adrive.loc[drive[away],'Time']
            else:
                if (hdrive.loc[drive[home],'Time'] > adrive.loc[drive[away],'Time']):
                    poss = 'H'
                    starttime = hdrive.loc[drive[home],'Time']
                else:
                    poss = 'A'
                    starttime = adrive.loc[drive[away],'Time']
        else:
            if (drive[home] <= hdrive.shape[0]):
                poss = 'H'
                starttime = hdrive.loc[drive[home],'Time']
            else:
                poss = 'A'
                starttime = adrive.loc[drive[away],'Time']
        rz = False   # Not in Red Zone
        bquar = df[team[poss]].loc[drive[team[poss]],'Quarter'] # Beginning quarter of drive
        if (df[team[poss]].loc[drive[team[poss]],'Length'] <= starttime): # Set time for next drive
            endtime = starttime - df[team[poss]].loc[drive[team[poss]],'Length']
            equar = bquar                                            # Ending quarter of drive
        else:    
            endtime = (timedelta(hours=0,minutes=15,seconds=0) -  # Assuming 15 minute quarters
                (df[team[poss]].loc[drive[team[poss]],'Length'] - starttime))
            equar = bquar+1                                            # Ending quarter of drive

        if (df[team[poss]].loc[drive[team[poss]],'Side'] != team[poss].upper()):
            if (df[team[poss]].loc[drive[team[poss]],'Marker'] <= 20):
                rz = True
            else:
                start = df[team[poss]].loc[drive[team[poss]],'Marker'] - 20
        else:
            start = 100 - df[team[poss]].loc[drive[team[poss]],'Marker'] - 20 # Other side of the field
        if not (rz):    # If starting outside Red Zone
            end = start - df[team[poss]].loc[drive[team[poss]],'Yds']   # Calculate end of drive
            if end <= 20: # If drive ends within 20 yards of red zone to the goal line check if goes in RZ
                found = False     # Look for beginning of drive
                while (((row+1) < pbp.shape[0]) & (not (found))):
                    if (pbp.loc[row,'Quarter'] < bquar):
                        row += 1
                    elif (pbp.loc[row,'Time'] <= starttime):
                        found = True
                    else:
                        row += 1
                while (not(stop)):
                    if ((pbp.loc[row,'Team'] != team[poss].upper()) & (pbp.loc[row,'Location'] <= 20)):
                        rz = True
                        stop = True
                    elif (pbp.loc[row,'Quarter'] > equar):
                        stop = True
                    elif (row+1) == pbp.shape[0]:                         # Failsafe in case of bad data
                        stop = True
                    elif ((pbp.loc[row,'Quarter'] == equar) & (pbp.loc[row,'Time'] <= endtime)):
                        stop = True
                    else:
                        row += 1
        if (rz):
            rz_att[team[poss]] += 1 # Increment Red Zone Attempts
            if df[team[poss]].loc[drive[team[poss]],'Result'] in ['Touchdown']:
                rz_conv[team[poss]] += 1  # If score, increment Red Zone successes 
        drive[team[poss]] += 1  # Increment drive number for team in question

    dfyear.loc[(away,gamecode),'RZ_Att'] = rz_att[away]
    dfyear.loc[(home,gamecode),'RZ_Att'] = rz_att[home]
    dfyear.loc[(away,gamecode),'RZ_Conv'] = rz_conv[away]
    dfyear.loc[(home,gamecode),'RZ_Conv'] = rz_conv[home]
    dfyear.loc[(away,gamecode),'RZ_Def_Att'] = rz_att[home]
    dfyear.loc[(home,gamecode),'RZ_Def_Att'] = rz_att[away]
    dfyear.loc[(away,gamecode),'RZ_Def_Conv'] = rz_conv[home]
    dfyear.loc[(home,gamecode),'RZ_Def_Conv'] = rz_conv[away]

for colu in dfyear.columns: dfyear.fillna(0,inplace=True)
intcols=['Points','Points_Opp','Yds_Off_Pass','Yds_Off_Rush','Yds_Def_Pass','Yds_Def_Rush','TD','TD_on_Def',
         'FG_Made','FG_Att','RZ_Conv','RZ_Att','RZ_Def_Conv','RZ_Def_Att','Plays','TO_Gained','TO_Lost',
         'Yds_Pen','Tackles_Loss']
fltcols=['Possession','Sacks_Def','Yds_per_Kickret','Yds_per_Puntret']
strcols=['Home/Away', 'Opponent']
for colu in intcols: dfyear[colu] = dfyear[colu].astype('int64')
for colu in fltcols: dfyear[colu] = dfyear[colu].astype('float64')
for colu in strcols: dfyear[colu] = dfyear[colu].astype('str')
print('End')

**Write *dfyear* to file**

In [ ]:
path = 'df_step2_'+year+'.pkl'
with open(path, 'wb') as f:
    pickle.dump(dfyear,f)